In [1]:
database = "T:/Occurrence_Records/Quick1.sqlite"
outFile = "T:/Occurrence_Records/random1.shp"
mode = "random"
make_file = True

In [2]:
from datetime import datetime
import sqlite3
import pandas as pd
import geopandas as gpd
import random
from shapely.geometry import Point

timestamp = datetime.now()

# Get the record coordinates as a data frame
records = (pd.read_sql("""SELECT * FROM occurrence_records;""",
                       con=sqlite3.connect(database))
                       .astype({'decimalLongitude': 'float',
                               'decimalLatitude': 'float',
                               'radius_m': 'float'}))

# Make a point geometry from coordinates
gdf = gpd.GeoDataFrame(records,
                       geometry=gpd.points_from_xy(records['decimalLongitude'],
                                                   records['decimalLatitude']))

# Set the coordinate reference system
gdf.crs={'init' :'epsg:4326'}

# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<  POINTS
# If user just wants shapefile of given coordinates (point method)
if mode == "points":
    if make_file == True:
        # Simply save points
        gdf.to_file(outFile)

    out = gdf

# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<  FOOTPRINTS
# If the user wants to identify record footprints (point-radius or shape method)
if mode == "footprints":
    # Reproject record coordinates to facilitate buffering
    feetprints = gdf.to_crs(epsg=5070)

    # Use point-radius method
    feetprints['footprint'] = feetprints.apply(lambda x: x.geometry.buffer(x.radius_m), axis=1)
    feetprints.set_geometry(col='footprint', inplace=True, drop=True)

    # Overwrite when footprint WKT is provided (shape method; careful with projection) FORTHCOMING
    if list(gdf['footprintWKT'].unique()) != ['nan']:
        print("ALERT! footprintWKT was provided but not used.")

    if make_file == True:
        # Save results as shapefile
        feetprints.to_file(outFile)

    out = feetprints
    

In [3]:
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<  RANDOM POINTS
# If the user wants a random point selected from within each footprint
if mode == "random":
    # Reproject record coordinates to facilitate buffering
    feetprints = gdf.to_crs(epsg=5070)
    
    # Use point-radius method
    feetprints['footprint'] = feetprints.apply(lambda x: x.geometry.buffer(x.radius_m), axis=1)
    feetprints.set_geometry(col='footprint', inplace=True, drop=True)

    # Overwrite when footprint WKT is provided (shape method; careful with projection) FORTHCOMING   
    if list(gdf['footprintWKT'].unique()) != ['nan']:
        print("ALERT! footprintWKT was provided but not used.")
        
    # FROM https://gis.stackexchange.com/questions/207731/generating-random-coordinates-in-multipolygon-in-python/207740#207740
    def generate_random(number, polygon):
        points = []
        minx, miny, maxx, maxy = polygon.bounds
        while len(points) < number:
            pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
            if polygon.contains(pnt):
                points.append(pnt)
        return points
    
    # 
    feetprints["random_points"] = feetprints["geometry"].apply(lambda x: generate_random(1, x)[0])
    
    if make_file == True:
        # Save results as shapefile
        feetprints.to_file(outFile)
    
    out=feetprints
    
print("Exported shapefile: " + str(datetime.now() - timestamp))

Index(['index', 'GBIF_download_doi', 'accessRights', 'basisOfRecord',
       'bibliographicCitation', 'collectionCode', 'coordinatePrecision',
       'coordinateUncertaintyInMeters', 'dataGeneralizations', 'datasetName',
       'decimalLatitude', 'decimalLongitude', 'detection_distance_m',
       'ebird_id', 'effort_distance_m', 'establishmentMeans', 'eventDate',
       'eventRemarks', 'filter_set_name', 'footprintSRS', 'footprintWKT',
       'gbif_id', 'general_remarks', 'geodeticDatum', 'georeferenceProtocol',
       'georeferenceRemarks', 'georeferenceVerificationStatus',
       'georeferencedBy', 'gps_accuracy_m', 'habitat',
       'identificationQualifier', 'identifiedBy', 'identifiedRemarks',
       'individualCount', 'informationWitheld', 'institutionID', 'issues',
       'license', 'locality', 'locationAccordingTo', 'locationRemarks',
       'modified', 'nominal_xy_precision', 'occurrenceRemarks',
       'occurrenceStatus', 'organismQuantity', 'organismQuantityType',
       'ra

TypeError: Cannot interpret '<geopandas.array.GeometryDtype object at 0x000001E925AA8C88>' as a data type